In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import time
from urllib.parse import urlparse, parse_qs
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import urllib.request
import pandas as pd

In [3]:
# url = "https://www.musinsa.com/app/styles/lists"
# style_list = [americancasual, casual, chic, dandy, formal, golf, romantic, street, gorpcore, highteen, minimal]
# style_url = f"https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type={americancasual}&brand=&tag_no=&display_cnt=60&list_kind=big&sort=NEWEST&page="

# 하나의 컴퓨터로 모든 스타일을 다 수집하면 오래 걸렸기에 여러대의 컴퓨터로 나누
style_list = ["street", "gorpcore", "highteen", "minimal"] 

all_info = []
driver = webdriver.Chrome()

for style in style_list:
    style_url = f"https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type={style}&brand=&tag_no=&display_cnt=60&list_kind=big&sort=NEWEST&page="
    driver.get(style_url)

    soup = BeautifulSoup(driver.page_source, "lxml")
    total_pages = int(driver.find_element(By.CSS_SELECTOR, "span.totalPagingNum").text)

    # 각 페이지 반복
    for page in range(1, total_pages + 1):
        if page == total_pages:
            driver.back()  # 뒤로가기
            continue  # 다음 모델로 넘어감
        url = f"{style_url}{page}" 
        driver.get(url)
        time.sleep(0.1)

        # 창 크기 최대로 키우기
        driver.maximize_window()

        # 성별 선택(남자) 클릭
        men = driver.find_elements(By.CSS_SELECTOR, "div.sc-pvystx-0.iRAZYo")
        men[0].click()

        time.sleep(0.1)

        # 이미지 요소를 찾을 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.style-list-item")))


        model_img_num = soup.select("div.style-list-thumbnail img")
        # image_style를 페이지에 존재하는 이미지 수 만큼 반복
        for i in range(len(model_img_num)): # 
            info = []
            stylename = driver.find_element(By.CSS_SELECTOR,"span.style-list-information__text")
            info.append(stylename.text)      
            image_style = driver.find_elements(By.CSS_SELECTOR, "li.style-list-item")
       
            if len(image_style) <= i:
                continue

    
            image_style[i].click() # 모델전신샷 클릭
            

            time.sleep(0.1)
            
            # 동영상 태그가 있는 경우 스킵
            if len(driver.find_elements(By.CSS_SELECTOR, "video")) > 0:
                driver.back()  # 뒤로가기
                continue  # 다음 모델로 넘어감
            
            # 모델 전체 착샷 이미지url info에 저장
            images = driver.find_elements(By.CSS_SELECTOR,"img.detail_img.slick-slide.slick-current.slick-active")
            for img in images:
                # 이미지 전체 주소
                img_url = img.get_attribute("src")
                # 모델컷 이미지 id
                model_id = img_url.split("/")[-2] # 범위로 수집하니 오류발생해서 id 수집 기준 코드 작성
                # 모델 id 값으로 이미지 저장
                urllib.request.urlretrieve(img_url, f"./image_formal/{style}_{model_id}.jpg")
                info.append(img_url)

                time.sleep(0.1)

            number = driver.find_elements(By.CSS_SELECTOR,"a.styling_img")

            # 모델이 입고 있는 옷들 id 6개까지만 추출
            for idx, save in enumerate(number):
                url = save.get_attribute('href')
                if idx < 6:
                    info.append(url[34:41])

                time.sleep(0.1)

            # info 리스트를 8개씩 분할하여 all_info에 추가
            all_info.extend([info[j:j+8] for j in range(0, len(info), 8)])

            driver.back()

driver.quit()

In [4]:
# 수집된 데이터 프레임으로 변환
all_info_df = pd.DataFrame(all_info)

In [5]:
all_info_df

,0,1,2,3,4,5,6,7
0,스트릿,https://image.msscdn.net/images/style/detail/4...,3950131,3907986,3941781,3833320,3392458,3966134
1,스트릿,https://image.msscdn.net/images/style/detail/4...,3965825,3959480,3339112,3739813,4002233,4038446
2,스트릿,https://image.msscdn.net/images/style/detail/4...,4026667,3856586,3406665,4033383,4062312,4028706
3,스트릿,https://image.msscdn.net/images/style/detail/4...,4033198,4025143,2879419,3984987,4027926,3844510
4,스트릿,https://image.msscdn.net/images/style/detail/4...,3436016,3798419,3989324,4035853,3970915,3973073
...,...,...,...,...,...,...,...,...
3685,고프코어,https://image.msscdn.net/images/style/detail/2...,2570026,2040290,None,None,None,None
3686,고프코어,https://image.msscdn.net/images/style/detail/2...,2352209,1952315,2060178,None,None,None
3687,고프코어,https://image.msscdn.net/images/style/detail/2...,2476116,2242778,2416590,1310113,None,None
3688,고프코어,https://image.msscdn.net/images/style/detail/2...,2410029,2242778,2231915,1762406,None,None


In [6]:
all_info_df[all_info_df.columns[0]].value_counts()

스트릿     3498
고프코어     192
Name: 0, dtype: int64

In [7]:
all_info_df.to_csv("./data/start_street_men_info.csv", index = False)